In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/poems100/poems.csv


In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from collections import defaultdict
from tqdm import tqdm


In [3]:
# Load dataset from CSV
def load_data(csv_path):
    df = pd.read_csv(csv_path)
    text = ' '.join(df['text'].tolist())  # Assuming 'poem' column has text
    return text

In [ ]:
# # load data is returning a single string
# text = load_data('/kaggle/input/poems100/poems.csv')

# print(text[0])

In [4]:
def tokenize(text):
    words = text.split()
    vocab = sorted(set(words))
    word_to_idx = {word: i for i, word in enumerate(vocab)}
    idx_to_word = {i: word for word, i in word_to_idx.items()}
    return words, vocab, word_to_idx, idx_to_word


In [ ]:
# words, vocab, word_to_idx, idx_to_word = tokenize(text)

# print(words[3])
# print(len(words))
# print(len(vocab))


# word is a lsit of words , vocab is a list of unique words word_to_idx is a dictionar maps each word to an index
# idx_to_word is a dictionar which is the reverse of word_to_idx

In [5]:
def encode_words(words, word_to_idx):
    return torch.tensor([word_to_idx[word] for word in words], dtype=torch.long)


In [ ]:
# encoded = one_hot_encode(words,len(vocab),word_to_idx)

# print(encoded[0])

In [6]:
# Define PyTorch Dataset class
class TextDataset(Dataset):
    def __init__(self, indexed_words, seq_length):
        self.inputs = torch.stack([indexed_words[i:i+seq_length] for i in range(len(indexed_words) - seq_length)])
        self.targets = indexed_words[seq_length:]
    
    def __len__(self):
        return len(self.inputs)
    
    def __getitem__(self, idx):
        return self.inputs[idx], self.targets[idx]

In [7]:
# Define RNN model with Embedding Layer
class SimpleRNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, output_size, dropout_rate=0.3):
        super(SimpleRNN, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(dropout_rate)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        x = self.embedding(x)
        out, _ = self.rnn(x)
        out = self.dropout(out[:, -1, :])
        out = self.fc(out)  # No softmax; CrossEntropyLoss applies it internally
        return out

In [8]:
# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
# Training loop
def train_model(model, train_loader, criterion, optimizer, epochs=25):
    model.to(device)  # Move model to GPU
    for epoch in range(epochs):
        total_loss = 0
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}", leave=False)
        for inputs, targets in progress_bar:
            inputs, targets = inputs.to(device), targets.to(device).long()  # Ensure LongTensor
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            progress_bar.set_postfix(loss=total_loss/len(train_loader))
        print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader)}")


In [ ]:
print(f"Using device: {device}")


In [10]:
csv_path = "/kaggle/input/poems100/poems.csv"  # Change this to your actual CSV file path
text = load_data(csv_path)
words, vocab, word_to_idx, idx_to_word = tokenize(text)
indexed_words = encode_words(words, word_to_idx)

seq_length = 10  # Adjusted sequence length for better training
batch_size = 32  # Increased batch size to fully utilize T4 GPU

dataset = TextDataset(indexed_words, seq_length)
train_size = int(0.9 * len(dataset))  # Increased training split
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

dataloader_train = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
dataloader_test = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)


In [11]:
input_size = len(vocab)
embedding_dim = 128  # Embedding size
hidden_size = 256  # Increased hidden size for better learning
output_size = len(vocab)

dropout_rate = 0.3  # Dropout to reduce overfitting
model = SimpleRNN(input_size, embedding_dim, hidden_size, output_size, dropout_rate).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005, weight_decay=1e-5)  # Added L2 regularization
train_model(model, dataloader_train, criterion, optimizer)

Epoch 1, Loss: 7.4868393088209215


Epoch 2, Loss: 6.75057561301637


Epoch 3, Loss: 6.25886596824931


Epoch 4, Loss: 5.734122224237727


Epoch 5, Loss: 5.174198654876358


...

Epoch 21, Loss: 0.4924088331560294


Epoch 22, Loss: 0.42885800671560326


Epoch 23, Loss: 0.37430697906462623


Epoch 24, Loss: 0.3318963312468995


Epoch 25, Loss: 0.2857505098499101


In [12]:
# Test loop for accuracy
def test_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device).long()  # Ensure LongTensor
            outputs = model(inputs)
            predicted = torch.argmax(outputs, dim=1)
            correct += (predicted == targets).sum().item()
            total += targets.size(0)
    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")

In [13]:
test_model(model, dataloader_test)

Test Accuracy: 6.96%
